## Import

In [ ]:
pip install torch==1.13.1

In [ ]:
pip install numpy==1.21.2

In [ ]:
pip install pandas==1.5.3

In [ ]:
pip install opencv-python==4.5.5.64

In [ ]:
pip install pytorchvideo==0.1.5

In [ ]:
pip install tqdm==4.62.2

In [ ]:
pip install conda==4.10.3

In [ ]:
pip install jupyterlab==2.3.2

In [ ]:
pip install nvidia-ml-py==11.495.46

In [1]:
import random
import pandas as pd
import numpy as np
import os


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 
import PIL
import scipy

from sklearn.metrics import accuracy_score
import cv2

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
CFG = {
    'VIDEO_LENGTH':50,
    'IMG_SIZE':128, 
    'EPOCHS':2, 
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':4,
    'SEED':41 
}

## Fixed RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Load

In [5]:
trainp="/data/train"
df = pd.read_csv('/data/train.csv')
df.head()

,sample_id,video_path,label
0,TRAIN_0000,/jupyter-pytorch/train/TRAIN_0000.mp4,7
1,TRAIN_0001,/jupyter-pytorch/train/TRAIN_0001.mp4,7
2,TRAIN_0002,/jupyter-pytorch/train/TRAIN_0002.mp4,0
3,TRAIN_0003,/jupyter-pytorch/train/TRAIN_0003.mp4,0
4,TRAIN_0004,/jupyter-pytorch/train/TRAIN_0004.mp4,1


## Data Analysis

In [6]:
L = df['label']

a = np.count_nonzero(L == 0)
print(f"crush\nYes: {len(L)-a}, No: {a}")

b = np.count_nonzero((0 < L)&(L < 7))
print(f"\nego-involve\nYes: {b}, No: {len(L)-b-a}")

c = np.count_nonzero((L == 1)|(L == 2)|(L == 7)|(L == 8))
d = np.count_nonzero((L == 3)|(L == 4)|(L == 9)|(L == 10))
print(f"\nweather\nNormal: {c}, Snowy: {d}, Rainy: {len(L)-(a+c+d)}")

e = np.count_nonzero(L%2!=0)
print(f"\ntiming\nDay: {e}, Night: {len(L)-(a+e)}")

crush
Yes: 915, No: 1783

ego-involve
Yes: 491, No: 424

weather
Normal: 716, Snowy: 129, Rainy: 70

timing
Day: 808, Night: 107


In [7]:
# outlier
ii = [104, 124, 146, 161, 165, 182, 187, 230, 244, 330, 387, 469, 471, 486, 548, 607, 651, 712, 717, 777, 825, 871, 961, 1005, 1068, 1084, 1113, 1184, 1207, 1220, 1318, 1421, 1475, 1499, 1537, 1565, 1612, 1630, 1714, 1854, 1903, 2084, 2289, 2292, 2445, 2610, 2657, 2666, 2678, 2695, 115, 789, 979, 1068, 1312, 1333, 1694, 1718, 1789, 2437]

In [8]:
crush = pd.read_csv('/data/train.csv')
# idx = []
for i in range(len(df)):
    crush["video_path"][i] = trainp+crush["video_path"][i][7:]
    if crush["label"][i] != 0: # Not YES
        crush["label"][i] = 1

In [9]:
crush = crush.drop(ii)
crush = crush.reset_index()
crush = crush.drop(columns=['index'])

In [10]:
a = np.count_nonzero(crush== 0)
print(f"crush\nYes: {len(crush)-a}, No: {a}")

crush
Yes: 905, No: 1734


In [11]:
ego = pd.read_csv('/data/train.csv')
for i in range(len(df)):
    ego["video_path"][i] = trainp+ego["video_path"][i][7:]

for i in range(len(ego)):
    if 1 <= ego["label"][i] <= 6: # YES
        ego["label"][i] = 1
    elif 7 <= ego["label"][i] <= 12: # NO
        ego["label"][i] = 2

In [12]:
ego = ego.drop(ii)
ego = ego.reset_index()
ego = ego.drop(columns=['index'])

In [13]:
a = np.count_nonzero(ego == 0)
b = np.count_nonzero(ego == 1)
c = np.count_nonzero(ego == 2)
print(a, b, c)

1734 483 422


In [14]:
timing = pd.read_csv('/data/train.csv')

idx = []
for i in range(len(timing)):
    if timing['label'][i] == 0:
        idx.append(i)

for i in range(len(df)):
    timing["video_path"][i] = trainp+timing["video_path"][i][7:]

for j in range(len(timing)):
    if timing["label"][j] != 0:
        if (timing["label"][j] % 2) == 0: # DAY
            timing["label"][j] = 1
        else:                             # NIGHT
            timing["label"][j] = 0

In [15]:
# label 0인 data hand labling
re_label = [1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1]

In [16]:
x = 0
for k in idx:
    timing['label'][k] = re_label[x]
    x += 1

In [17]:
timing = timing.drop(ii)
timing = timing.reset_index()
timing = timing.drop(columns=['index'])

In [18]:
e = np.count_nonzero(timing==1)
print(f"\ntiming\nDay: {e}, Night: {len(L)-e}")


timing
Day: 1042, Night: 1656


In [19]:
weather = pd.read_csv('/data/train.csv')
for i in range(len(df)):
    weather["video_path"][i] = trainp+weather["video_path"][i][7:]

for j in range(len(weather)):
    k = weather["label"][j]
    if k != 0:
        if (1<=k<=2) or (7<=k<=8): # Normal
            weather["label"][j] = 0
        elif (3<=k<=4) or (9<=k<=10): # Snowy
            weather["label"][j] = 1
        else:                        # Rainy
            weather["label"][j] = 2

In [20]:
# weather dataset 전체 hand labeling
all = [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0, 1, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 2, 0, 0, 2, 1, 2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 1, 1, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 1, 2, 0, 1, 0, 1, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 1, 0, 2, 0, 2, 0, 1, 1, 0, 2, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 2, 0, 2, 0, 0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 2, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 1, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 1, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 1, 0, 0, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 2, 0, 0, 2, 2, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 2, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 0, 0, 2, 0, 1, 2, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 1, 2, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 2, 1, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 1, 0, 2, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 2, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 1, 1, 1, 1, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 2, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 1, 2, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 1, 0, 0, 2, 2, 0, 1, 0, 1, 1, 0, 1, 1, 1, 2, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 1, 2, 2, 1, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 2, 0, 0, 2, 1, 1, 2, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 2, 2, 0, 0, 2, 1, 1, 0, 1, 0, 0, 0, 2, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 1, 0, 2, 2, 0]

In [21]:
for k in range(len(all)):
    weather['label'][k] = all[k]

In [22]:
weather = weather.drop(ii)
weather = weather.reset_index()
weather = weather.drop(columns=['index'])

In [23]:
A = np.count_nonzero(weather==0)
B = np.count_nonzero(weather==1)
C = np.count_nonzero(weather==2)
print(f"\nweather\nNormal: {A}, Snowy: {B}, Rainy: {C}")


weather
Normal: 1926, Snowy: 313, Rainy: 400


In [24]:
test = pd.read_csv('/data/test.csv')
testp = "/data/test"
for i in range(len(test)):
    test['video_path'][i] = testp+test['video_path'][i][6:]

## Train / Validation Split

In [25]:
c_train, c_val, __, _ = train_test_split(crush, crush['label'], test_size=0.1, random_state=CFG['SEED'])
e_train, e_val, __, _ = train_test_split(ego, ego['label'], test_size=0.1, random_state=CFG['SEED'])
t_train, t_val, __, _ = train_test_split(timing, timing['label'], test_size=0.2, random_state=CFG['SEED'])
w_train, w_val, __, _ = train_test_split(weather, weather['label'], test_size=0.2, random_state=CFG['SEED'])

## CustomDataset

In [26]:
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize

In [27]:

def cv2_to_pil(image):
    return Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

class CustomDataset1(Dataset):
    def __init__(self, video_path_list, label_list, tfms):
        self.video_path_list = video_path_list
        self.label_list = label_list
        self.tfms = tfms
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            img = cv2.resize(img,(CFG["IMG_SIZE"], CFG["IMG_SIZE"]), interpolation = cv2.INTER_AREA)
            img = cv2_to_pil(img)
            img = (self.tfms(img))
            frames.append(img)
        frames = torch.from_numpy(np.stack(frames))
        return torch.FloatTensor(np.array(frames)).permute(1,0,2,3)

train_transform = Compose([
#                  Resize((128, 128), interpolation = ), 
#                  CenterCrop(112), 
                 ToTensor(), 
                 Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989])])

In [28]:
class CustomDataset2(Dataset):
    def __init__(self, video_path_list, label_list, tfms):
        self.video_path_list = video_path_list
        self.label_list = label_list
        self.tfms = tfms
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            img = cv2.resize(img,(200, 200), interpolation = cv2.INTER_AREA)
            img = cv2_to_pil(img)
            img = (self.tfms(img))
            frames.append(img)
        frames = torch.from_numpy(np.stack(frames))
        return torch.FloatTensor(np.array(frames)).permute(1,0,2,3)

train_transform = Compose([
#                  Resize((128, 128), interpolation = ), 
#                  CenterCrop(112), 
                 ToTensor(), 
                 Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989])])

In [29]:
pip install fvcore

Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install pytorchvideo

Note: you may need to restart the kernel to use updated packages.


In [31]:
# # crash data
# c_train_dataset = CustomDataset1(c_train['video_path'].values, c_train['label'].values,tfms = train_transform)
# c_train_loader = DataLoader(c_train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=4)
# c_val_dataset = CustomDataset1(c_val['video_path'].values, c_val['label'].values,tfms = train_transform)
# c_val_loader = DataLoader(c_val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=4)


# ego-involve data
e_train_dataset =CustomDataset1(e_train['video_path'].values, e_train['label'].values,tfms = train_transform)
e_train_loader = DataLoader(e_train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=4)
e_val_dataset = CustomDataset1(e_val['video_path'].values, e_val['label'].values,tfms = train_transform)
e_val_loader = DataLoader(e_val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=4)

# timing data
t_train_dataset = CustomDataset2(t_train['video_path'].values, t_train['label'].values,tfms = train_transform)
t_train_loader = DataLoader(t_train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=4)
t_val_dataset = CustomDataset2(t_val['video_path'].values, t_val['label'].values,tfms = train_transform)
t_val_loader = DataLoader(t_val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=4)

# weather data
w_train_dataset = CustomDataset2(w_train['video_path'].values, w_train['label'].values,tfms = train_transform)
w_train_loader = DataLoader(w_train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=4)
w_val_dataset = CustomDataset2(w_val['video_path'].values, w_val['label'].values,tfms = train_transform)
w_val_loader = DataLoader(w_val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=4)

In [32]:
# test data (img size: 128)
test_dataset = CustomDataset1(test['video_path'].values, None, tfms = train_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=4)
# test data (img size: 200)
w_test_dataset = CustomDataset2(test['video_path'].values, None, tfms = train_transform)
w_test_loader = DataLoader(w_test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=4)

## Model Define

In [33]:
import torch.nn.functional as F

In [34]:
class BaseModel2(nn.Module): # ego
    def __init__(self, num_classes=3):
        super(BaseModel2, self).__init__()
        self.feature_extract = models.video.r3d_18(pretrained=True)
        self.f1 = nn.Linear(400, 256)
        self.f3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.feature_extract(x)
        x = x.view(batch_size, -1)
        x = F.relu(self.f1(x))
        x = self.dropout(x)
        x = self.f3(x)
        return x

In [35]:
class BaseModel3(nn.Module): # timing
    def __init__(self, num_classes=2):
        super(BaseModel3, self).__init__()
        self.feature_extract = models.video.r3d_18(pretrained=True)
        self.f1 = nn.Linear(400, 256)
        self.f3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.feature_extract(x)
        x = x.view(batch_size, -1)
        x = F.relu(self.f1(x))
        x = self.dropout(x)
        x = self.f3(x)
        return x

In [36]:
class BaseModel4(nn.Module): # weather
    def __init__(self, num_classes=3):
        super(BaseModel4, self).__init__()
        self.feature_extract = models.video.r3d_18(pretrained=True)
        self.f1 = nn.Linear(400, 256)
        self.f3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.feature_extract(x)
        x = x.view(batch_size, -1)
        x = F.relu(self.f1(x))
        x = self.dropout(x)
        x = self.f3(x)
        return x

## Train

In [37]:
def train(model, optimizer, train_loader, val_loader, scheduler, device, best_val_score): # epoch: 1
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    Iter = round(len(e_train)/4)
    best_val_score = best_val_score
    best_model = model
    
    for epoch in range(1, 2):
        model.train() 
        train_loss = []
        bat = 1
        for videos, labels in tqdm(iter(train_loader)): 
            
            videos = videos.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            output = model(videos)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()
            bat += 1
            train_loss.append(loss.item())
            print(f"batch {bat}/{Iter}", end='\r')
            
        _val_loss, _val_score, _f1_, acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : {_val_score} [{_f1_:.5f}], Val Acc: [{acc:.3f}]')
        
        if scheduler is not None:
            scheduler.step(_f1_)

        if best_val_score < _f1_:
            best_val_score = _f1_
            best_model = model
    return best_model, best_val_score

In [38]:
def train2(model, optimizer, train_loader, val_loader, scheduler,  device, best_val_score): # epoch: 2
    criterion = nn.CrossEntropyLoss().to(device)
    Iter = round(len(e_train)/4)
    best_val_score = best_val_score
    best_model = model
    
    for epoch in range(1,3):
        model.train()
        train_loss = []
        bat = 1
        for videos, labels in tqdm(iter(train_loader)): 
            
            videos = videos.to(device)
            labels = labels.to(device) 
            optimizer.zero_grad()
            output = model(videos)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            bat += 1
            train_loss.append(loss.item())
            print(f"batch {bat}/{Iter}", end='\r')
            
        _val_loss, _val_score, _f1_, acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : {_val_score} [{_f1_:.5f}], Val Acc: [{acc:.3f}]')
        
        if scheduler is not None:
            scheduler.step(_f1_)

        if best_val_score < _f1_:
            best_val_score = _f1_
            best_model = model
    return best_model, best_val_score

In [39]:
def train3(model, optimizer, train_loader, val_loader, scheduler, device, best_val_score): # epoch: 3
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    Iter = round(len(e_train)/4)
    best_val_score = best_val_score
    best_model = model
    
    for epoch in range(1, 4):
        model.train()
        train_loss = []
        bat = 1
        for videos, labels in tqdm(iter(train_loader)): 
            
            videos = videos.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            output = model(videos)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            bat += 1
            train_loss.append(loss.item())
            print(f"batch {bat}/{Iter}", end='\r')
    
        _val_loss, _val_score, _f1_, acc= validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : {_val_score} [{_f1_:.5f}], Val Acc: [{acc:.3f}]')
        
        if scheduler is not None:
            scheduler.step(_f1_)

        if best_val_score < _f1_:
            best_val_score = _f1_
            best_model = model
    return best_model, best_val_score

In [40]:
def train5(model, optimizer, train_loader, val_loader, scheduler, device, best_val_score): # epoch: 5
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    Iter = round(len(e_train)/4)
    best_val_score = best_val_score
    best_model = model
    
    for epoch in range(1, 6):
        model.train()
        train_loss = []
        bat = 1
        for videos, labels in tqdm(iter(train_loader)): 
            
            videos = videos.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            output = model(videos)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            bat += 1
            train_loss.append(loss.item())
            print(f"batch {bat}/{Iter}", end='\r')
    
        _val_loss, _val_score, _f1_, acc= validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : {_val_score} [{_f1_:.5f}], Val Acc: [{acc:.3f}]')
        
        if scheduler is not None:
            scheduler.step(_f1_)

        if best_val_score < _f1_:
            best_val_score = _f1_
            best_model = model
    return best_model, best_val_score

In [41]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    bat = 1
    Iter = round(len(e_val)/4)
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)

            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            _val_loss = np.mean(val_loss)
            bat += 1
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
            if bat%10 == 0:
                print(f"batch {bat}/{Iter} Val Loss: {_val_loss}", end='\r')
            
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average=None)
    _f1_ = f1_score(trues, preds, average='macro')
    acc = accuracy_score(trues, preds)
    return _val_loss, _val_score, _f1_, acc

In [42]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
        
            logit = model(videos)
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [44]:
import torch
# torch version: 1.13.1+cu117
print(torch.__version__)

1.13.1+cu117


## weather

In [70]:
ml = BaseModel4()
ml.eval()
optimizer = torch.optim.Adam(params = ml.parameters(), lr = 1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, 
                                                       patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)
infer_ml, best_val_score = train(ml, optimizer, w_train_loader, w_val_loader, scheduler, device, 0)

  0%|          | 0/528 [00:00<?, ?it/s]

  0%|          | 0/132 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.58646] Val Loss : [0.47647] Val F1 : [0.90518331 0.60240964 0.66666667] [0.72475], Val Acc: [0.835]


In [71]:
optimizer = torch.optim.Adam(params = infer_ml.parameters(), lr = 5e-5, weight_decay=1e-5)
infer_ml, best_val_score = train(infer_ml, optimizer, w_train_loader, w_val_loader, None, device, best_val_score)

  0%|          | 0/528 [00:00<?, ?it/s]

  0%|          | 0/132 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.38547] Val Loss : [0.32124] Val F1 : [0.93       0.68852459 0.74626866] [0.78826], Val Acc: [0.879]


In [72]:
optimizer = torch.optim.Adam(params = infer_ml.parameters(), lr = 1e-5, weight_decay=1e-5)
infer_ml, best_val_score = train(infer_ml, optimizer, w_train_loader, w_val_loader, None, device, best_val_score)

  0%|          | 0/528 [00:00<?, ?it/s]

  0%|          | 0/132 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.24458] Val Loss : [0.29932] Val F1 : [0.9398773  0.72268908 0.75409836] [0.80555], Val Acc: [0.894]


In [73]:
optimizer = torch.optim.Adam(params = infer_ml.parameters(), lr = 1e-6)
infer_ml, best_val_score = train(infer_ml, optimizer, w_train_loader, w_val_loader, None, device, best_val_score)

  0%|          | 0/528 [00:00<?, ?it/s]

  0%|          | 0/132 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.18312] Val Loss : [0.30183] Val F1 : [0.94132335 0.73170732 0.78787879] [0.82030], Val Acc: [0.898]


In [74]:
w_preds = inference(infer_ml, test_loader, device)

  0%|          | 0/450 [00:00<?, ?it/s]

In [79]:
w_submit = pd.read_csv('/data/sample_submission.csv')
w_submit['label'] = w_preds
w_submit.to_csv('/data/Weather.csv', index=False)

In [80]:
torch.save(infer_ml.state_dict(), '/data/Weather.pth')